In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.data

import torchvision
from torchvision import transforms

import matplotlib.pyplot as plt

import curvature
from curvature import losses
from curvature import utils
#from curvature.methods.lanczosopt import LanczosSGD

import time

from tqdm import tqdm_notebook

%matplotlib inline

In [2]:
train_set = torchvision.datasets.MNIST('/home/tgaripov/projects/data/mnist/', train=True, download=True,
    transform=transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.1307,), (0.3081,))
    ]))

test_set = torchvision.datasets.MNIST('/home/tgaripov/projects/data/mnist/', train=False, download=True,
    transform=transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.1307,), (0.3081,))
    ]))

In [3]:
train_set[0][1].dtype

torch.int64

In [4]:
import torch
from torch.optim.optimizer import Optimizer, required
from gpytorch.utils.lanczos import lanczos_tridiag


class LanczosSGD(Optimizer):
    """
        based on sgd optimizer.
    """

    def __init__(self, params, hess_vec_closure, lanczos_steps=5, beta=1.0, lr=required, dampening=0,
                 weight_decay=0):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        self.hess_vec_closure = hess_vec_closure
        self.lanczos_steps = lanczos_steps
        self.beta = beta

        defaults = dict(lr=lr, dampening=dampening, weight_decay=weight_decay)
        super(LanczosSGD, self).__init__(params, defaults)


    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        num_parametrs = sum([p.numel() for group in self.param_groups for p in group['params']])

        grad_list = []
        for group in self.param_groups:
            weight_decay = group['weight_decay']
            for p in group['params']:
                if p.grad is not None:
                    grad_wd = p.grad.data + weight_decay * p.data
                    grad_list.append(grad_wd.cpu().view(-1))
        grad = torch.cat(grad_list)

        Q, T = lanczos_tridiag(
            self.hess_vec_closure,
            self.lanczos_steps,
            dtype=torch.float32,
            device='cpu',
            matrix_shape=(num_parametrs, num_parametrs)
        )

        eigvals, T_eigvects = T.eig(eigenvectors=True)
        eigvals = eigvals[:, 0]
        eigvects = T_eigvects.t() @ Q.t()

        overlaps = torch.mm(eigvects, grad.view(-1, 1))
        lambdas = torch.abs(eigvals.view(-1, 1))
        weights = 1.0 / (lambdas + self.beta) #- 1.0 / self.beta
        overlaps.mul_(weights)

        final = torch.mm(eigvects.t(), overlaps).view(-1) #+ grad / self.beta

        offset = 0
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = final[offset:offset+p.numel()].view_as(p).to(p.device)

                p.data.add_(-group['lr'], d_p)
                offset += p.numel()

        return loss


In [5]:
class HessVecProduct(object):
    def __init__(self, batch_size, model, criterion):
        self.batch_size = batch_size
        X_batch = torch.zeros(self.batch_size, 784)
        y_batch = torch.zeros(self.batch_size, dtype=torch.int64)
        self.dataset = torch.utils.data.TensorDataset(X_batch, y_batch)
        self.loader = torch.utils.data.DataLoader(self.dataset, batch_size=self.batch_size)
        self.model = model
        self.criterion = criterion
        self.iters = 0
        self.timestamp = time.time()
        
    def set_batch(self, X, y):    
        self.dataset.tensors = (X.clone().detach(), y.clone().detach())        

    def __call__(self, vector):
        start_time = time.time()
        output = utils.hess_vec(vector, self.loader, self.model, self.criterion, cuda=False)
        time_diff = time.time() - start_time

        self.iters += 1
        #print('Iter %d. Time: %.2f' % (self.iters, time_diff))
        return output.cpu().unsqueeze(1)


In [6]:
def train(model, device, train_loader, optimizer, criterion):
    model.train()    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        if isinstance(optimizer, LanczosSGD):
            optimizer.hess_vec_closure.set_batch(data, target)
        loss, output, stats = criterion(model, data, target)        
        loss.backward()
        optimizer.step()                
            
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            loss, output, stats = criterion(model, data, target)
            
            test_loss += loss.item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

In [7]:
class LinearModel(torch.nn.Module):
    
    def __init__(self):
        super(LinearModel, self).__init__()
        self.layer = torch.nn.Linear(784, 10, bias=True)
        
    def forward(self, x):
        return self.layer(x.view(-1, 784))

In [8]:
train_loader = torch.utils.data.DataLoader(
    torch.utils.data.Subset(train_set, torch.randperm(len(train_set))[:100]),
    batch_size=50,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=50,
    shuffle=False
)

In [11]:


#optimizer = torch.optim.SGD(model.parameters(), 0.01)
#productor = HessVecProduct(50, model, losses.cross_entropy)
#optimizer = LanczosSGD(model.parameters(), productor, lr=0.01, lanczos_steps=10, beta=0.1)

In [33]:
def run_lanczos(model, T, batch_size=50, epochs=50, lr=1.0, lanczos_steps=20, beta=1.0, dataset_seed=1):
    
    rng = np.random.RandomState(dataset_seed)
    order = torch.tensor(rng.permutation(len(train_set)))
        
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(train_set, order[:T]),
        batch_size=batch_size,
        shuffle=True
    )

    test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=batch_size,
        shuffle=False
    )
    
    device = torch.device('cpu')    

    productor = HessVecProduct(batch_size, model, losses.cross_entropy)
    optimizer = LanczosSGD(model.parameters(), productor, lr=lr, lanczos_steps=lanczos_steps, beta=beta)
    
    results = {name: [] for name in ['train_loss', 'train_acc', 'test_loss', 'test_acc']}
    
    train_loss, train_acc = test(model, device, train_loader, losses.cross_entropy)
    test_loss, test_acc = test(model, device, test_loader, losses.cross_entropy)
    print('Train loss: %.4f Train acc: %.2f Test loss: %.4f Test acc: %.2f' % 
          (train_loss, train_acc, test_loss, test_acc))

    results['train_loss'].append(train_loss)
    results['test_loss'].append(test_loss)
    results['train_acc'].append(train_acc)
    results['test_acc'].append(test_acc)
    
    for i in range(epochs):
        train(model, device, train_loader, optimizer, losses.cross_entropy)
        train_loss, train_acc = test(model, device, train_loader, losses.cross_entropy)
        test_loss, test_acc = test(model, device, test_loader, losses.cross_entropy)
        print('Train loss: %.4f Train acc: %.2f Test loss: %.4f Test acc: %.2f' % 
              (train_loss, train_acc, test_loss, test_acc))
        
        results['train_loss'].append(train_loss)
        results['test_loss'].append(test_loss)
        results['train_acc'].append(train_acc)
        results['train_acc'].append(train_acc)
        
    return results

In [13]:
def run_sgd(model, T, batch_size=50, epochs=50, lr=0.01, dataset_seed=1):
    
    rng = np.random.RandomState(dataset_seed)
    order = torch.tensor(rng.permutation(len(train_set)))
        
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.Subset(train_set, order[:T]),
        batch_size=batch_size,
        shuffle=True
    )

    test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=batch_size,
        shuffle=False
    )
    
    device = torch.device('cpu')    
    
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    results = {name: [] for name in ['train_loss', 'train_acc', 'test_loss', 'test_acc']}
    
    train_loss, train_acc = test(model, device, train_loader, losses.cross_entropy)
    test_loss, test_acc = test(model, device, test_loader, losses.cross_entropy)
    print('Train loss: %.4f Train acc: %.2f Test loss: %.4f Test acc: %.2f' % 
          (train_loss, train_acc, test_loss, test_acc))

    results['train_loss'].append(train_loss)
    results['test_loss'].append(test_loss)
    results['train_acc'].append(train_acc)
    results['test_acc'].append(test_acc)
    
    for i in range(epochs):
        train(model, device, train_loader, optimizer, losses.cross_entropy)
        train_loss, train_acc = test(model, device, train_loader, losses.cross_entropy)
        test_loss, test_acc = test(model, device, test_loader, losses.cross_entropy)
        print('Train loss: %.4f Train acc: %.2f Test loss: %.4f Test acc: %.2f' % 
              (train_loss, train_acc, test_loss, test_acc))
        
        results['train_loss'].append(train_loss)
        results['test_loss'].append(test_loss)
        results['train_acc'].append(train_acc)
        results['train_acc'].append(train_acc)
        
    return results

In [51]:
device = torch.device('cpu')
model_1 = LinearModel()
model_2 = LinearModel()
model_2.load_state_dict(model_1.state_dict())



In [52]:
lanczos_res = run_lanczos(model_1, T=100, batch_size=100, lr=1.0, beta=1.0, lanczos_steps=50, epochs=40)

Train loss: 0.0269 Train acc: 4.00 Test loss: 0.0252 Test acc: 8.41
Train loss: 0.0165 Train acc: 42.00 Test loss: 0.0190 Test acc: 30.16
Train loss: 0.0112 Train acc: 65.00 Test loss: 0.0148 Test acc: 47.07
Train loss: 0.0084 Train acc: 76.00 Test loss: 0.0126 Test acc: 59.31
Train loss: 0.0066 Train acc: 80.00 Test loss: 0.0110 Test acc: 65.80
Train loss: 0.0053 Train acc: 86.00 Test loss: 0.0106 Test acc: 66.40
Train loss: 0.0049 Train acc: 87.00 Test loss: 0.0103 Test acc: 67.44
Train loss: 0.0042 Train acc: 88.00 Test loss: 0.0098 Test acc: 69.01
Train loss: 0.0036 Train acc: 94.00 Test loss: 0.0094 Test acc: 70.40
Train loss: 0.0032 Train acc: 94.00 Test loss: 0.0090 Test acc: 71.41
Train loss: 0.0028 Train acc: 95.00 Test loss: 0.0089 Test acc: 71.66
Train loss: 0.0023 Train acc: 96.00 Test loss: 0.0085 Test acc: 72.98
Train loss: 0.0020 Train acc: 97.00 Test loss: 0.0083 Test acc: 73.51
Train loss: 0.0016 Train acc: 98.00 Test loss: 0.0083 Test acc: 73.12
Train loss: 0.0015 Tra

In [47]:
sgd_res = run_sgd(model_2, T=50, epochs=200, lr=0.1)

Train loss: 0.0482 Train acc: 6.00 Test loss: 0.0498 Test acc: 7.45
Train loss: 0.0306 Train acc: 46.00 Test loss: 0.0556 Test acc: 22.52
Train loss: 0.0393 Train acc: 50.00 Test loss: 0.0434 Test acc: 36.64
Train loss: 0.0184 Train acc: 68.00 Test loss: 0.0458 Test acc: 40.57
Train loss: 0.0067 Train acc: 94.00 Test loss: 0.0273 Test acc: 53.07
Train loss: 0.0050 Train acc: 92.00 Test loss: 0.0264 Test acc: 57.02
Train loss: 0.0036 Train acc: 98.00 Test loss: 0.0271 Test acc: 54.49
Train loss: 0.0029 Train acc: 100.00 Test loss: 0.0251 Test acc: 58.09
Train loss: 0.0023 Train acc: 100.00 Test loss: 0.0255 Test acc: 56.07
Train loss: 0.0020 Train acc: 100.00 Test loss: 0.0249 Test acc: 57.38
Train loss: 0.0018 Train acc: 100.00 Test loss: 0.0250 Test acc: 57.02
Train loss: 0.0016 Train acc: 100.00 Test loss: 0.0249 Test acc: 57.26
Train loss: 0.0015 Train acc: 100.00 Test loss: 0.0249 Test acc: 57.39
Train loss: 0.0014 Train acc: 100.00 Test loss: 0.0249 Test acc: 57.49
Train loss: 0.0

Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0270 Test acc: 58.77
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0270 Test acc: 58.77
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0270 Test acc: 58.76
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0270 Test acc: 58.77
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0271 Test acc: 58.77
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0271 Test acc: 58.76
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0271 Test acc: 58.76
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0271 Test acc: 58.77
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0271 Test acc: 58.77
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0271 Test acc: 58.77
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0271 Test acc: 58.75
Train loss: 0.0002 Train acc: 100.00 Test loss: 0.0271 Test acc: 58.75
Train loss: 0.0001 Train acc: 100.00 Test loss: 0.0272 Test acc: 58.75
Train loss: 0.0001 Train acc: 100.00 Test loss: 0.0272 Test acc: 58.76
Train 

KeyboardInterrupt: 

In [96]:
import sys

def eval_model(model, train_loader, test_loader):
    train_loss, train_acc = test(model, device, train_loader, losses.cross_entropy)
    test_loss, test_acc = test(model, device, test_loader, losses.cross_entropy)
    
    return {
        'train_loss': train_loss,
        'train_acc': train_acc,
        'test_loss': test_loss,
        'test_acc': test_acc
    }
    
def parallel_runs(opt_configs, T=100, batch_size=None, dataset_seed=1, model_seed=1, epochs=200):
    
    rng = np.random.RandomState(dataset_seed)
        
    if batch_size is None:
        batch_size = T
    
    while True:
        order = torch.tensor(rng.permutation(len(train_set)))
        subset = torch.utils.data.Subset(train_set, order[:T])        
        bc = np.bincount(subset.dataset.train_labels[subset.indices])        
        if bc.size == 10 and np.min(bc) > 1:
            break
                
    train_loader = torch.utils.data.DataLoader(
        subset,
        batch_size=batch_size,
        shuffle=True
    )

    test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=200,
        shuffle=False
    )
    
    torch.manual_seed(model_seed)    
    init_model = LinearModel()
    
    K = len(opt_configs)        
    
    models = []
    for k in range(K):
        models.append(LinearModel())
        models[-1].load_state_dict(init_model.state_dict())
        
    optimizers = []
    for k in range(K):
        config = opt_configs[k]
        print(config['name'])
        if config['type'] == LanczosSGD:
            productor = HessVecProduct(batch_size, models[k], losses.cross_entropy)
            optimizer = LanczosSGD(models[k].parameters(), productor, **config['params'])
        else:        
            optimizer = config['type'](models[k].parameters(), **config['params'])
        optimizers.append(optimizer)
            
    results = [{name: [] for name in ['train_loss', 'train_acc', 'test_loss', 'test_acc']} for k in range(K)]
    for k in range(K):
        results[k]['name'] = opt_configs[k]['name']
    
    for k in range(K):
        eval_res = eval_model(models[k], train_loader, test_loader)
        for key, val in eval_res.items():
            results[k][key].append(val)
            
    for k in range(K):
        print('%10s Train loss: %.5f Train acc: %.2f Test loss: %.5f Test acc: %.2f' % 
             (opt_configs[k]['name'], results[k]['train_loss'][-1], results[k]['train_acc'][-1],
              results[k]['test_loss'][-1], results[k]['test_acc'][-1]))
    
    for epoch in range(epochs):            
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            for k in range(K):
                optimizer = optimizers[k]
                model = models[k]                
                model.train()
                optimizer.zero_grad()
                if isinstance(optimizer, LanczosSGD):
                    optimizer.hess_vec_closure.set_batch(data, target)
                loss, _, _ = losses.cross_entropy(model, data, target)        
                loss.backward()
                optimizer.step()                        
                
                eval_res = eval_model(model, train_loader, test_loader)
                for key, val in eval_res.items():
                    results[k][key].append(val)
            sys.stdout.write('.')
            sys.stdout.flush()
        print()
                    
        print('Epoch %d' % (epoch + 1))
        
        for k in range(K):
            print('%10s Train loss: %.5f Train acc: %.2f Test loss: %.5f Test acc: %.2f' % 
                 (opt_configs[k]['name'], results[k]['train_loss'][-1], results[k]['train_acc'][-1],
                  results[k]['test_loss'][-1], results[k]['test_acc'][-1]))
                    
    return results

In [123]:
opt_configs = [
    {
        'name': 'Lanczos',
        'type': LanczosSGD,
        'params': {
            'lr': 1.0,
            'beta': 0.5,
            'lanczos_steps': 100,            
        }
    },
    {
        'name': 'SGD_02',
        'type': torch.optim.SGD,
        'params': {
            'lr': 0.2,                        
        }
    },
    {
        'name': 'SGD_01',
        'type': torch.optim.SGD,
        'params': {
            'lr': 0.1,                        
        }
    },
    {
        'name': 'SGD_002',
        'type': torch.optim.SGD,
        'params': {
            'lr': 0.02,                        
        }
    }
]

In [124]:
import pickle
T = 50
for data_seed in [1, 2, 3]:
    for model_seed in [1, 2, 3]:
        print('NEW RUN')
        results = parallel_runs(opt_configs, T=T, epochs=100, dataset_seed=data_seed, model_seed=model_seed)
        with open('./run_%d_d%d_m%d.pkl' % (T, data_seed, model_seed), 'wb') as handle:
            pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

NEW RUN
Lanczos
SGD_02
SGD_01
SGD_002
   Lanczos Train loss: 0.04796 Train acc: 16.00 Test loss: 0.01143 Test acc: 20.61
    SGD_02 Train loss: 0.04796 Train acc: 16.00 Test loss: 0.01143 Test acc: 20.61
    SGD_01 Train loss: 0.04796 Train acc: 16.00 Test loss: 0.01143 Test acc: 20.61
   SGD_002 Train loss: 0.04796 Train acc: 16.00 Test loss: 0.01143 Test acc: 20.61
.
Epoch 1
   Lanczos Train loss: 0.02882 Train acc: 54.00 Test loss: 0.01270 Test acc: 25.92
    SGD_02 Train loss: 0.08390 Train acc: 30.00 Test loss: 0.03359 Test acc: 14.01
    SGD_01 Train loss: 0.04108 Train acc: 36.00 Test loss: 0.01636 Test acc: 15.75
   SGD_002 Train loss: 0.03650 Train acc: 46.00 Test loss: 0.01036 Test acc: 27.17
.
Epoch 2
   Lanczos Train loss: 0.01871 Train acc: 68.00 Test loss: 0.00858 Test acc: 41.33
    SGD_02 Train loss: 0.14891 Train acc: 60.00 Test loss: 0.02824 Test acc: 35.08
    SGD_01 Train loss: 0.06171 Train acc: 62.00 Test loss: 0.01347 Test acc: 43.38
   SGD_002 Train loss: 0.0303

.
Epoch 24
   Lanczos Train loss: 0.00005 Train acc: 100.00 Test loss: 0.00732 Test acc: 60.52
    SGD_02 Train loss: 0.00018 Train acc: 100.00 Test loss: 0.00707 Test acc: 62.36
    SGD_01 Train loss: 0.00077 Train acc: 100.00 Test loss: 0.00581 Test acc: 62.36
   SGD_002 Train loss: 0.00544 Train acc: 100.00 Test loss: 0.00619 Test acc: 60.94
.
Epoch 25
   Lanczos Train loss: 0.00004 Train acc: 100.00 Test loss: 0.00734 Test acc: 60.57
    SGD_02 Train loss: 0.00017 Train acc: 100.00 Test loss: 0.00706 Test acc: 62.39
    SGD_01 Train loss: 0.00074 Train acc: 100.00 Test loss: 0.00582 Test acc: 62.34
   SGD_002 Train loss: 0.00521 Train acc: 100.00 Test loss: 0.00616 Test acc: 60.98
.
Epoch 26
   Lanczos Train loss: 0.00004 Train acc: 100.00 Test loss: 0.00736 Test acc: 60.60
    SGD_02 Train loss: 0.00017 Train acc: 100.00 Test loss: 0.00706 Test acc: 62.41
    SGD_01 Train loss: 0.00071 Train acc: 100.00 Test loss: 0.00583 Test acc: 62.31
   SGD_002 Train loss: 0.00499 Train acc: 1

.
Epoch 48
   Lanczos Train loss: 0.00002 Train acc: 100.00 Test loss: 0.00766 Test acc: 60.54
    SGD_02 Train loss: 0.00011 Train acc: 100.00 Test loss: 0.00709 Test acc: 62.51
    SGD_01 Train loss: 0.00039 Train acc: 100.00 Test loss: 0.00600 Test acc: 62.06
   SGD_002 Train loss: 0.00251 Train acc: 100.00 Test loss: 0.00588 Test acc: 61.58
.
Epoch 49
   Lanczos Train loss: 0.00002 Train acc: 100.00 Test loss: 0.00767 Test acc: 60.55
    SGD_02 Train loss: 0.00011 Train acc: 100.00 Test loss: 0.00710 Test acc: 62.51
    SGD_01 Train loss: 0.00038 Train acc: 100.00 Test loss: 0.00601 Test acc: 62.06
   SGD_002 Train loss: 0.00246 Train acc: 100.00 Test loss: 0.00588 Test acc: 61.59
.
Epoch 50
   Lanczos Train loss: 0.00002 Train acc: 100.00 Test loss: 0.00768 Test acc: 60.56
    SGD_02 Train loss: 0.00011 Train acc: 100.00 Test loss: 0.00710 Test acc: 62.48
    SGD_01 Train loss: 0.00038 Train acc: 100.00 Test loss: 0.00602 Test acc: 62.06
   SGD_002 Train loss: 0.00240 Train acc: 1

.
Epoch 72
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00787 Test acc: 60.62
    SGD_02 Train loss: 0.00008 Train acc: 100.00 Test loss: 0.00715 Test acc: 62.43
    SGD_01 Train loss: 0.00026 Train acc: 100.00 Test loss: 0.00615 Test acc: 61.93
   SGD_002 Train loss: 0.00159 Train acc: 100.00 Test loss: 0.00584 Test acc: 61.67
.
Epoch 73
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00788 Test acc: 60.61
    SGD_02 Train loss: 0.00008 Train acc: 100.00 Test loss: 0.00715 Test acc: 62.43
    SGD_01 Train loss: 0.00026 Train acc: 100.00 Test loss: 0.00615 Test acc: 61.91
   SGD_002 Train loss: 0.00157 Train acc: 100.00 Test loss: 0.00584 Test acc: 61.64
.
Epoch 74
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00789 Test acc: 60.61
    SGD_02 Train loss: 0.00008 Train acc: 100.00 Test loss: 0.00716 Test acc: 62.43
    SGD_01 Train loss: 0.00026 Train acc: 100.00 Test loss: 0.00616 Test acc: 61.90
   SGD_002 Train loss: 0.00155 Train acc: 1

.
Epoch 96
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00803 Test acc: 60.65
    SGD_02 Train loss: 0.00007 Train acc: 100.00 Test loss: 0.00721 Test acc: 62.41
    SGD_01 Train loss: 0.00020 Train acc: 100.00 Test loss: 0.00626 Test acc: 61.74
   SGD_002 Train loss: 0.00116 Train acc: 100.00 Test loss: 0.00586 Test acc: 61.56
.
Epoch 97
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00803 Test acc: 60.64
    SGD_02 Train loss: 0.00007 Train acc: 100.00 Test loss: 0.00721 Test acc: 62.43
    SGD_01 Train loss: 0.00020 Train acc: 100.00 Test loss: 0.00626 Test acc: 61.74
   SGD_002 Train loss: 0.00115 Train acc: 100.00 Test loss: 0.00586 Test acc: 61.56
.
Epoch 98
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00804 Test acc: 60.64
    SGD_02 Train loss: 0.00007 Train acc: 100.00 Test loss: 0.00722 Test acc: 62.41
    SGD_01 Train loss: 0.00019 Train acc: 100.00 Test loss: 0.00626 Test acc: 61.74
   SGD_002 Train loss: 0.00113 Train acc: 1

...

.
Epoch 85
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00795 Test acc: 60.48
    SGD_02 Train loss: 0.00008 Train acc: 100.00 Test loss: 0.00679 Test acc: 60.96
    SGD_01 Train loss: 0.00023 Train acc: 100.00 Test loss: 0.00653 Test acc: 60.10
   SGD_002 Train loss: 0.00132 Train acc: 100.00 Test loss: 0.00625 Test acc: 59.01
.
Epoch 86
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00795 Test acc: 60.49
    SGD_02 Train loss: 0.00008 Train acc: 100.00 Test loss: 0.00679 Test acc: 60.96
    SGD_01 Train loss: 0.00023 Train acc: 100.00 Test loss: 0.00653 Test acc: 60.11
   SGD_002 Train loss: 0.00131 Train acc: 100.00 Test loss: 0.00625 Test acc: 59.01
.
Epoch 87
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00796 Test acc: 60.49
    SGD_02 Train loss: 0.00008 Train acc: 100.00 Test loss: 0.00680 Test acc: 60.96
    SGD_01 Train loss: 0.00023 Train acc: 100.00 Test loss: 0.00654 Test acc: 60.11
   SGD_002 Train loss: 0.00129 Train acc: 1

.
Epoch 8
   Lanczos Train loss: 0.00023 Train acc: 100.00 Test loss: 0.00664 Test acc: 59.84
    SGD_02 Train loss: 0.00045 Train acc: 100.00 Test loss: 0.00678 Test acc: 57.24
    SGD_01 Train loss: 0.00239 Train acc: 100.00 Test loss: 0.00615 Test acc: 58.43
   SGD_002 Train loss: 0.01643 Train acc: 92.00 Test loss: 0.00823 Test acc: 46.92
.
Epoch 9
   Lanczos Train loss: 0.00019 Train acc: 100.00 Test loss: 0.00669 Test acc: 59.95
    SGD_02 Train loss: 0.00041 Train acc: 100.00 Test loss: 0.00674 Test acc: 57.58
    SGD_01 Train loss: 0.00213 Train acc: 100.00 Test loss: 0.00614 Test acc: 58.61
   SGD_002 Train loss: 0.01491 Train acc: 98.00 Test loss: 0.00800 Test acc: 48.75
.
Epoch 10
   Lanczos Train loss: 0.00016 Train acc: 100.00 Test loss: 0.00674 Test acc: 60.05
    SGD_02 Train loss: 0.00038 Train acc: 100.00 Test loss: 0.00671 Test acc: 57.70
    SGD_01 Train loss: 0.00192 Train acc: 100.00 Test loss: 0.00613 Test acc: 58.81
   SGD_002 Train loss: 0.01359 Train acc: 98.00

.
Epoch 32
   Lanczos Train loss: 0.00004 Train acc: 100.00 Test loss: 0.00730 Test acc: 60.59
    SGD_02 Train loss: 0.00017 Train acc: 100.00 Test loss: 0.00668 Test acc: 59.18
    SGD_01 Train loss: 0.00061 Train acc: 100.00 Test loss: 0.00621 Test acc: 59.61
   SGD_002 Train loss: 0.00397 Train acc: 100.00 Test loss: 0.00643 Test acc: 57.49
.
Epoch 33
   Lanczos Train loss: 0.00004 Train acc: 100.00 Test loss: 0.00732 Test acc: 60.61
    SGD_02 Train loss: 0.00017 Train acc: 100.00 Test loss: 0.00669 Test acc: 59.19
    SGD_01 Train loss: 0.00059 Train acc: 100.00 Test loss: 0.00621 Test acc: 59.60
   SGD_002 Train loss: 0.00383 Train acc: 100.00 Test loss: 0.00641 Test acc: 57.62
.
Epoch 34
   Lanczos Train loss: 0.00003 Train acc: 100.00 Test loss: 0.00733 Test acc: 60.62
    SGD_02 Train loss: 0.00017 Train acc: 100.00 Test loss: 0.00669 Test acc: 59.20
    SGD_01 Train loss: 0.00057 Train acc: 100.00 Test loss: 0.00622 Test acc: 59.62
   SGD_002 Train loss: 0.00370 Train acc: 1

.
Epoch 56
   Lanczos Train loss: 0.00002 Train acc: 100.00 Test loss: 0.00758 Test acc: 60.78
    SGD_02 Train loss: 0.00012 Train acc: 100.00 Test loss: 0.00680 Test acc: 59.53
    SGD_01 Train loss: 0.00035 Train acc: 100.00 Test loss: 0.00633 Test acc: 59.99
   SGD_002 Train loss: 0.00209 Train acc: 100.00 Test loss: 0.00623 Test acc: 58.47
.
Epoch 57
   Lanczos Train loss: 0.00002 Train acc: 100.00 Test loss: 0.00759 Test acc: 60.79
    SGD_02 Train loss: 0.00011 Train acc: 100.00 Test loss: 0.00681 Test acc: 59.55
    SGD_01 Train loss: 0.00034 Train acc: 100.00 Test loss: 0.00633 Test acc: 60.02
   SGD_002 Train loss: 0.00205 Train acc: 100.00 Test loss: 0.00622 Test acc: 58.53
.
Epoch 58
   Lanczos Train loss: 0.00002 Train acc: 100.00 Test loss: 0.00760 Test acc: 60.76
    SGD_02 Train loss: 0.00011 Train acc: 100.00 Test loss: 0.00681 Test acc: 59.58
    SGD_01 Train loss: 0.00034 Train acc: 100.00 Test loss: 0.00634 Test acc: 60.05
   SGD_002 Train loss: 0.00201 Train acc: 1

.
Epoch 80
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00777 Test acc: 60.84
    SGD_02 Train loss: 0.00009 Train acc: 100.00 Test loss: 0.00691 Test acc: 59.69
    SGD_01 Train loss: 0.00024 Train acc: 100.00 Test loss: 0.00643 Test acc: 60.20
   SGD_002 Train loss: 0.00140 Train acc: 100.00 Test loss: 0.00619 Test acc: 58.90
.
Epoch 81
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00777 Test acc: 60.83
    SGD_02 Train loss: 0.00009 Train acc: 100.00 Test loss: 0.00691 Test acc: 59.68
    SGD_01 Train loss: 0.00024 Train acc: 100.00 Test loss: 0.00643 Test acc: 60.20
   SGD_002 Train loss: 0.00138 Train acc: 100.00 Test loss: 0.00618 Test acc: 58.89
.
Epoch 82
   Lanczos Train loss: 0.00001 Train acc: 100.00 Test loss: 0.00778 Test acc: 60.83
    SGD_02 Train loss: 0.00009 Train acc: 100.00 Test loss: 0.00692 Test acc: 59.70
    SGD_01 Train loss: 0.00024 Train acc: 100.00 Test loss: 0.00644 Test acc: 60.20
   SGD_002 Train loss: 0.00136 Train acc: 1